In [1]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
import os

In [2]:
graph = Neo4jGraph(
    url="YOUR_URL", username="USER_NAME", password="PASSWORD"
)

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "OPEN_API_KEY"

In [4]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def run_query(self, query, parameters=None):
        try:
            with self.driver.session() as session:
                return session.run(query, parameters)
        except CypherSyntaxError as e:
            print("Cypher 쿼리 구문 오류가 발생했습니다:", e.message)
        except Exception as e:
            print("예상치 못한 오류가 발생했습니다:", str(e))

In [5]:
# 원본
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
If you receive a question about data that is not stored in neo4j, please answer in Korean that you are not sure about the information.
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Construct a Cypher statement that can retrieve all requested information in a single query.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
#If a question comes in in Korean, translate it into English.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Find the name part of the node, but if there is detail content in the node, please tell us the content of detail as a result.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [6]:
def run_query_with_exception_handling(question):
    try:
        result = chain.run(question)
        
        if not result or result == "I don't know the answer.":
            print('요청하신 정보를 찾을 수 없거나 데이터베이스에 없습니다.')
        else:
            print(result)
            
    except Exception as e:
        print(f'오류가 발생했습니다: {str(e)}')

In [18]:
chain.run("홍성캠퍼스의 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Campus {name:"홍성캠퍼스"})-[:LOCATED_AT]->(a:Address)
RETURN a.name AS address
Full Context:
[{'address': '충남 홍성군 홍성읍 대학1길 66'}]

> Finished chain.


'충남 홍성군 홍성읍 대학1길 66'

In [19]:
chain.run("수강철회 신청서는 어떻게 작성해?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CourseWithdrawal)-[:HAS_WITHDRAWAL_PROCEDURE_DOCUMENT]->(wd:WithdrawalDetail)
RETURN wd.detail AS WithdrawalProcedureDocument
Full Context:
[{'WithdrawalProcedureDocument': '홈페이지 http://home.chungwoon.ac.kr/ → 청운알림 → 서식자료실\r\n→ 학사서식 → 49번(수강과목 철회 신청서)'}]

> Finished chain.


'수강철회 신청서는 홈페이지 http://home.chungwoon.ac.kr/ 에서 청운알림 → 서식자료실 → 학사서식 → 49번(수강과목 철회 신청서)를 참고하여 작성하시면 됩니다.'

In [20]:
chain.run("이클래스 웹페이지 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Cwuwebpage)-[:OFFERS_EClass]->(w:Webpage)
RETURN w.detail AS webpageAddress
Full Context:
[{'webpageAddress': 'https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl'}]

> Finished chain.


'이클래스 웹페이지 주소는 https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl 입니다.'

In [21]:
chain.run("eclass 웹페이지 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Cwuwebpage)-[:OFFERS_EClass]->(w:Webpage)
RETURN w.detail AS eclassWebpageAddress
Full Context:
[{'eclassWebpageAddress': 'https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl'}]

> Finished chain.


'eclass 웹페이지 주소는 https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl 입니다.'

In [22]:
chain.run("컴퓨터공학과에 대해 설명해줘!")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Department {name:"컴퓨터공학과"})-[:HAS_DESCRIPTION]->(desc:Description)
RETURN desc.name AS Description
Full Context:
[{'Description': '컴퓨터공학과에서는 컴퓨터 시스템 및 소프트웨어 개발 능력과 현장 실무능력을 지닌 전문 엔지니어를 양성하고 있습니다.학과 특성을 지향하는 학술 동아리에 모든 학년이 함께 활동하고 있고, IT관련 전문 지식을 습득하고 관련 분야의 국제공인자격증 등을 취득하고 있습니다.Cisco와 Java 학술동아리는 매년 CCNA(네트워크 국제공인자격증)와 SCJP(자바프로그래머 국제공인자격증)를 취득하고 있으며, CNAP(Cisco Networking Academy Program) 프로그램을 운영하고 있습니다.소프트웨어 개발, 서버시스템 모바일 등의 프로그램을 산학협력과 실무중심으로 교육하여 매우 높은 취업률달성에 노력하고 있습니다. 실무경력이 풍부한 교수진들이 최신IT시설을 갖춘 다양한 교육시설과 최고 수준의 연구 환경을 바탕으로 현장 실습형 교육을 제공합니다.'}]

> Finished chain.


'컴퓨터공학과에서는 컴퓨터 시스템 및 소프트웨어 개발 능력과 현장 실무능력을 지닌 전문 엔지니어를 양성하고 있습니다. 학과 특성을 지향하는 학술 동아리에 모든 학년이 함께 활동하고 있고, IT관련 전문 지식을 습득하고 관련 분야의 국제공인자격증 등을 취득하고 있습니다. Cisco와 Java 학술동아리는 매년 CCNA(네트워크 국제공인자격증)와 SCJP(자바프로그래머 국제공인자격증)를 취득하고 있으며, CNAP(Cisco Networking Academy Program) 프로그램을 운영하고 있습니다. 소프트웨어 개발, 서버시스템 모바일 등의 프로그램을 산학협력과 실무중심으로 교육하여 매우 높은 취업률달성에 노력하고 있습니다. 실무경력이 풍부한 교수진들이 최신IT시설을 갖춘 다양한 교육시설과 최고 수준의 연구 환경을 바탕으로 현장 실습형 교육을 제공합니다.'

In [23]:
chain.run("학부생의 정기주차 요금은 얼마야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:SubscriptionParking)-[:HAS_UNDERGRADUATE_FEE]->(f:SubscriptionFee)
RETURN f.name AS UndergraduateParkingFee
Full Context:
[{'UndergraduateParkingFee': '5000원/월 혹은 20,000원/학기'}]

> Finished chain.


'5000원/월 혹은 20,000원/학기입니다.'

In [24]:
chain.run("인천캠퍼스의 주소와 홍성캠퍼스의 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (i:Campus)-[:LOCATED_AT]->(a:Address)
WHERE i.name = "인천캠퍼스" OR i.name = "홍성캠퍼스"
RETURN i.name, a.name as address
Full Context:
[{'i.name': '인천캠퍼스', 'address': '인천광역시 미추홀구 숙골로 113(도화동)'}, {'i.name': '홍성캠퍼스', 'address': '충남 홍성군 홍성읍 대학1길 66'}]

> Finished chain.


'인천캠퍼스의 주소는 인천광역시 미추홀구 숙골로 113(도화동)이고, 홍성캠퍼스의 주소는 충남 홍성군 홍성읍 대학1길 66입니다.'

In [25]:
chain.run("강의평가는 어떻게 활용되니?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:CourseEvaluation)-[:UTILIZES_EVALUATION_USAGE]->(ce:CourseEvaluationDetail)
RETURN ce.detail AS EvaluationUsage
Full Context:
[{'EvaluationUsage': '수업 진행에 대한 학생들의 피드백을 확인함으로써 수업 개선 정보로 활용, 교수님 업적평가 및 다음 학기 수업개설에 중요한 자료로 활용, 학과 및 우리 대학 수업의 질을 높이는 데 활용'}]

> Finished chain.


'수업 진행에 대한 학생들의 피드백을 확인함으로써 수업 개선 정보로 활용, 교수님 업적평가 및 다음 학기 수업개설에 중요한 자료로 활용, 학과 및 우리 대학 수업의 질을 높이는 데 활용합니다.'

In [26]:
chain.run("식당은 어디에 위치해 있니?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant)-[:LOCATED_AT]->(a:Address)
RETURN r.name, a.name as location
Full Context:
[]

> Finished chain.


"I don't know the answer."

In [ ]:
run_query_with_exception_handling("식당은 어디에 위치해 있니?")